<a href="https://colab.research.google.com/github/CarlosVasquezVelasco/DataVis/blob/main/FCF_Script2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install nest_asyncio
%pip install yfinance --upgrade --no-cache-dir 
%pip install pandas
%pip install bs4
%pip install requests 
%pip install selenium
#%pip install sklearn
%pip install streamlit --quiet
%pip install pyngrok==4.1.1 --quiet
%pip install graphics.py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1257 sha256=e2f79245d0dfa338ce6a17403251ddd024f3dbdf01a3fbe40c75ff92f902029e
  Stored in directory: /root/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
Successfully built bs4
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Operation cancelled by user
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 

In [ ]:
%pip install scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import re 
import json
import csv 
import time 
from bs4 import BeautifulSoup
from io import StringIO
import requests
import os
import pandas as pd 
import nest_asyncio
import yfinance as yf
from datetime import datetime
import lxml
from lxml import html
import numpy as np
import sklearn
from sklearn.tree import DecisionTreeClassifier
import streamlit as st

ModuleNotFoundError: ignored

In [ ]:
from pyngrok import ngrok
from google.colab import drive

In [ ]:
%%writefile my_app.py

In [ ]:
#headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:109.0) Gecko/20100101 Firefox/109.0"}
#url = "https://finance.yahoo.com/quote/ATAI"
#r = requests.get(url)

#print(r.status_code)

In [ ]:
symbol = 'MELI'

url = 'https://finance.yahoo.com/quote/' + symbol + '/balance-sheet?p=' + symbol
headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9',
    'Cache-Control': 'max-age=0',
    'Connection': 'close',
    'DNT': '1', # Do Not Track Request Header 
    'Pragma': 'no-cache',
    'Referrer': 'https://google.com',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36'
}
page = requests.get(url, headers = headers)
tree = html.fromstring(page.content)
tree.xpath("//h1/text()")

In [ ]:
table_rows = tree.xpath("//div[contains(@class, 'D(tbr)')]")

assert len(table_rows) > 0

parsed_rows = []

for table_row in table_rows:
    parsed_row = []
    el = table_row.xpath("./div")
    
    none_count = 0
    
    for rs in el:
        try:
            (text,) = rs.xpath('.//span/text()[1]')
            parsed_row.append(text)
        except ValueError:
            parsed_row.append(np.NaN)
            none_count += 1

    if (none_count < 4):
        parsed_rows.append(parsed_row)

df = pd.DataFrame(parsed_rows)
df

In [ ]:
df = pd.DataFrame(parsed_rows)
df = df.set_index(0) # Set the index to the first column: 'Period Ending'.
df = df.transpose() # Transpose the DataFrame, so that our header contains the account names

# Rename the "Breakdown" column to "Date"
cols = list(df.columns)
cols[0] = 'Date'
df = df.set_axis(cols, axis='columns', inplace=False)

df

In [ ]:
df.dtypes

In [ ]:
numeric_columns = list(df.columns)[1::] # Take all columns, except the first (which is the 'Date' column)

for column_name in numeric_columns:
    df[column_name] = df[column_name].str.replace(',', '') # Remove the thousands separator
    df[column_name] = df[column_name].astype(np.float64) # Convert the column to float64

df.dtypes

In [ ]:
# FINANCIAL SCRAPING FORMULAS #
def get_page(url):
    # Set up the request headers that we're going to use, to simulate
    # a request by the Chrome browser. Simulating a request from a browser
    # is generally good practice when building a scraper
    headers = {
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'en-US,en;q=0.9',
        'Cache-Control': 'max-age=0',
        'Connection': 'close',
        'DNT': '1', # Do Not Track Request Header 
        'Pragma': 'no-cache',
        'Referrer': 'https://google.com',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36'
    }

    return requests.get(url, headers=headers)

def parse_rows(table_rows):
    parsed_rows = []

    for table_row in table_rows:
        parsed_row = []
        el = table_row.xpath("./div")

        none_count = 0

        for rs in el:
            try:
                (text,) = rs.xpath('.//span/text()[1]')
                parsed_row.append(text)
            except ValueError:
                parsed_row.append(np.NaN)
                none_count += 1

        if (none_count < 4):
            parsed_rows.append(parsed_row)
            
    return pd.DataFrame(parsed_rows)

def clean_data(df):
    df = df.set_index(0) # Set the index to the first column: 'Period Ending'.
    df = df.transpose() # Transpose the DataFrame, so that our header contains the account names
    
    # Rename the "Breakdown" column to "Date"
    cols = list(df.columns)
    cols[0] = 'Date'
    df = df.set_axis(cols, axis='columns', inplace=False)
    
    numeric_columns = list(df.columns)[1::] # Take all columns, except the first (which is the 'Date' column)

    for column_index in range(1, len(df.columns)): # Take all columns, except the first (which is the 'Date' column)
        df.iloc[:,column_index] = df.iloc[:,column_index].str.replace(',', '') # Remove the thousands separator
        df.iloc[:,column_index] = df.iloc[:,column_index].astype(np.float64) # Convert the column to float64
        
    return df

def scrape_table(url):
    # Fetch the page that we're going to parse
    page = get_page(url);

    # Parse the page with LXML, so that we can start doing some XPATH queries
    # to extract the data that we want
    tree = html.fromstring(page.content)

    # Fetch all div elements which have class 'D(tbr)'
    table_rows = tree.xpath("//div[contains(@class, 'D(tbr)')]")
    
    # Ensure that some table rows are found; if none are found, then it's possible
    # that Yahoo Finance has changed their page layout, or have detected
    # that you're scraping the page.
    assert len(table_rows) > 0
    
    df = parse_rows(table_rows)
    df = clean_data(df)
        
    return df

In [ ]:
#symbol = "MELI"
#scrape_table('https://finance.yahoo.com/quote/' + symbol + 'key-statistics?p=' + symbol)

In [ ]:
#symbol = 'MELI'
df_balance_sheet = scrape_table('https://finance.yahoo.com/quote/' + symbol + '/balance-sheet?p=' + symbol)
df_balance_sheet

In [ ]:
scrape_table('https://finance.yahoo.com/quote/' + symbol + '/financials?p=' + symbol)

In [ ]:
#MELI = yf.Ticker('MELI')
# MELI.get_shares_full(start="2022-01-01", end=None)[-1]

# Gets the most recent shares outstanding by last trading date 
yf.Ticker("AAPL").get_shares_full(start="2022-01-01", end=None)[-1]

In [ ]:
def get_free_cashflow(symbol):
    #cf_table = pd.read_html(f'https://finance.yahoo.com/quote/{symbol}/cash-flow?p={symbol}')[0]
    cf_table = scrape_table('https://finance.yahoo.com/quote/' + symbol + '/cash-flow?p=' + symbol)
    table_FCF = cf_table.iloc[[0, 1, 2, 3, 4],[0 ,-1]]
    table_FCF2 =  table_FCF.iloc[1:, 1:]

    freeCF = []
    # Iterate over each row in the dataframe
    for index, row in table_FCF2.iterrows():
        # Iterate over each cell in the row and append its value to the list
        for value in row.values:
            freeCF.append(value)

    return freeCF

    

In [ ]:
freeCF=get_free_cashflow("ORCL")
freeCF[-1]

In [ ]:
symbol = "ORCL"
cf_table = scrape_table('https://finance.yahoo.com/quote/' + symbol + '/cash-flow?p=' + symbol)
table_FCF = cf_table.iloc[[0, 1, 2, 3, 4],[0 ,-1]]
table_FCF2 =  table_FCF.iloc[1:, 1:]
print(table_FCF2)

In [ ]:
def get_fairvalue(symbol):
    # Get free cash flow
    freeCF = get_free_cashflow(symbol)

    # Get outstanding shares
    ticker = yf.Ticker(symbol)
    outstandingshares = ticker.get_shares_full(start="2022-01-01", end=None)[-1]
    

    # Assumptions:
    required_rate = 0.07
    perpetual_rate = 0.02
    cfGrowth = 0.03
    years = [1, 2, 3, 4]

    futureFCF = []
    discountfactor = []
    discountedFCF = []

    # Terminal value calculation using Perpetuity Method 
    terminalvalue = (freeCF[0] * (1 + perpetual_rate)) / (required_rate - perpetual_rate)

    # Discounting For Loop
    for year in years:
        futureFCF.append(freeCF[-1] * (1 + cfGrowth) ** year)
        discountfactor.append((1 + required_rate) ** year)
        #discountedFCF.append(futureFCF[-1] / discountfactor[-1])


    for i in range(0, len(years)):
      discountedFCF.append(futureFCF[i]/discountfactor[i])

    discountedterminalvalue = terminalvalue / (1+required_rate)**4
    discountedFCF.append(discountedterminalvalue)

    todaysvalue = sum(discountedFCF)
    # Intrinsic Value Per Share Calculation
    fairvalue = todaysvalue*1000 / outstandingshares

    return fairvalue

    #return futureFCF
    
    print("The fair value is ${}".format(round(fairvalue,2)))
    #print(futureFCF)



In [ ]:
get_fairvalue("AAPL")

In [ ]:
def get_ticker(name):

    ticker = yf.Ticker(name)  
    return ticker

In [ ]:
######### MAIN BODY ##########
st.title('Fair Value Analysis Tool')
st.subheader("by Carlos Vasquez-Velasco")

# Containers help separate content. 
# Make sure to assign relevant names to distinguish containers
description = st.container()
description.write("""The objective of this Webapp is to provide informative data and value-based analysis of equities for the retail investor.""")

######### SIDEBAR MENU ########
st.sidebar.header("Honors By Contract")

######### MAIN BODY ##########
ticker_name = st.text_input("Enter ticker to analyze")
ticker = get_ticker(ticker_name)

if len(ticker_name) >= 1: 
  projectedPrice = get_fairvalue(ticker_name)
  

In [ ]:
######## COMPANY SUMMARY ########

# detailed summary on Ticker
if st.checkbox('Show company summary'):

    st.subheader('Company summary on: ' + str(ticker.info['longName']))
    st.write(ticker.info['longBusinessSummary']) 

######## RAW DATA DATAFRAME ########

## Make options inside checkbox bubbles a dropdown menu instead of more checkbox objects.

# Raw data for Ticker
if st.checkbox('Show raw data'):

    if len(ticker_name) < 1:
        st.warning("Please enter a ticker to analyze")

    # fetches the Fair Value of the Stock 
    if len(ticker_name) >= 1:
      company = get_fairvalue(ticker_name)

      st.write(company)
      
      submitted = st.form_submit_button("Submit")
      if submitted:
        st.success("Loading data!")

In [ ]:
# FCF VALUATION #

if st.checkbox('Conduct a Free Cash Flow Valuation', symbol):
  fcf_Valuation = get_fairvalue(symbol)

In [ ]:
!ngrok authtoken 2POFa91LcItBpbPVBoqM1qT1xDB_Fixjv9P2Rxrmy3DR5R4Q

In [ ]:
!nohup streamlit my_app.py &
url = ngrok.connect(port = '8501')
print("This is the URL: ", url)

In [ ]:
!streamlit run my_app.py

In [ ]:
%pip install graphics.py

In [ ]:
def get_company_description(ticker):
    url = f"https://finance.yahoo.com/quote/{ticker}"
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        start = response.text.find('\"longBusinessSummary\":\"') + 24
        end = response.text.find('\",\"city\"')
        return response.text[start:end]
    else:
        return "Error: Unable to retrieve company description"


In [ ]:
def get_companyProfile(ticker):
    """
    Retrieves the company profile of a given ticker symbol from Yahoo Finance.

    Parameters:
    ticker (str): The ticker symbol of the company.

    Returns:
    dict: A dictionary containing the company profile information.
    """
    try:
        company = yf.Ticker(ticker)
        info = company.info
        return info
    except Exception as e:
        print(f"An error occurred: {e}")


In [ ]:
def userInterface():
  
  enterTicker = str(input("Enter the ticker symbol: "))
  profile = get_companyProfile(enterTicker)
  companyProfile = profile['longBusinessSummary']
  analysisResult = get_fairvalue(enterTicker)
  print(enterTicker, companyProfile)
  print("The fair value of", enterTicker, " is: ", analysisResult)
  


In [ ]:
userInterface()